In [24]:
module LP

using NNLS
import Base: convert

type PNNLSProblem{T}
    A::Matrix{T}
    B::Matrix{T}
    c::Vector{T}
end

type LinearProgram{T}
    f::Vector{T}
    G::Matrix{T}
    W::Vector{T}
end

type NNLSProblem{T}
    A::Matrix{T}
    b::Vector{T}
end


function convert{T1, T2}(::Type{NNLSProblem{T1}}, p::PNNLSProblem{T2})
    A = convert(Matrix{T1}, p.A)
    B = convert(Matrix{T1}, p.B)
    c = convert(Vector{T1}, p.c)
    Abar = A - B * (B \ A)
    bbar = c - B * (B \ c)
#     @assert Abar ≈ (I - p.B * pinv(p.B)) * p.A
#     @assert bbar ≈ (I - p.B * pinv(p.B)) * p.c
    NNLSProblem{T1}(Abar, bbar)
end

convert{T}(::Type{NNLSProblem}, p::PNNLSProblem{T}) = convert(NNLSProblem{T}, p)

function convert{T1, T2}(::Type{PNNLSProblem{T1}}, lp::LinearProgram{T2})
    q, n = size(lp.G)
    A = [lp.W'       zeros(1, q)
         zeros(q, q) eye(q)
         lp.G'       zeros(n, q)]
    B = [lp.f'
         lp.G
         zeros(n, n)]
    c = vcat(0, lp.W, -lp.f)
    PNNLSProblem{T1}(A, B, c)
end

convert{T}(::Type{PNNLSProblem}, lp::LinearProgram{T}) = convert(PNNLSProblem{T}, lp)

function solve(p::NNLSProblem)
    work = NNLSWorkspace(p.A, p.b)
    nnls!(work)
    @show work.rnorm
    @show work.zz
    @show work.w
    @show work.nsetp
    @show size(work.QA)
    @show work.Qb
    work.x
end

function solve(p::PNNLSProblem)
    v = solve(convert(NNLSProblem, p))
    u = -p.B \ (p.A * v - p.c)
#     u = -pinv(p.B) * (p.A * v - p.c)
    v, u
end

function solve(p::LinearProgram)
    v, u = solve(convert(PNNLSProblem, p))
    y = v[1:length(p.W)]
    s = v[(length(p.W) + 1):end]
    y, s, u
end

end

LP

In [25]:
f = ones(2)
G = [1. 0
0 1
-1 0
0 -1]
W = [1., 1, 1, 1]
lp = LP.LinearProgram(f, G, W)
LP.solve(lp)

work.rnorm = 2.2443315815740113e-16
work.zz = [1.0,2.0,1.0,2.0,2.57517e-16,-2.20447e-16,-4.21091e-17]
work.w = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
work.nsetp = 5
size(work.QA) = (7,8)
work.Qb = [-0.816497,-1.52753,0.881917,-1.49071,-2.30331e-16,-2.20447e-16,-4.21091e-17]


([0.0,2.57517e-16,1.0,1.0],[2.0,2.0,0.0,0.0],[-1.0,-1.0])

In [26]:
f = ones(2)
G = [1. 0
0 1
-1 0]
W = [1., 1, 1]
lp = LP.LinearProgram(f, G, W)
LP.solve(lp)

work.rnorm = 1.0
work.zz = [2.0,1.0,1.0,2.77556e-16,5.55112e-17,-1.0]
work.w = [-3.08149e-32,-1.0,0.0,0.0,0.0,-4.62223e-32]
work.nsetp = 3
size(work.QA) = (6,6)
work.Qb = [1.54919,-0.866025,-0.5,2.77556e-16,5.55112e-17,-1.0]


([0.0,0.0,1.0],[2.0,1.0,0.0],[-1.0,-3.9968e-16])

In [27]:
f = ones(2)
G = [1. 0
0 1
-1 0
0 -1]
W = [1., 1, -1.0001, -1.0001]
lp = LP.LinearProgram(f, G, W)
LP.solve(lp)

work.rnorm = 0.00011180228089137442
work.zz = [0.999975,0.999975,-5.34018e-5,-4.65982e-5,-4.65982e-5,-1.51374e-5,-7.12466e-5]
work.w = [-2.49975e-9,-2.49975e-9,0.0,0.0,-6.24988e-5,-6.24988e-5,-3.75012e-5,-3.75012e-5]
work.nsetp = 2
size(work.QA) = (7,8)
work.Qb = [1.63306,-1.15469,-5.34018e-5,-4.65982e-5,-4.65982e-5,-1.51374e-5,-7.12466e-5]


([0.0,0.0,0.999975,0.999975],[0.0,0.0,0.0,0.0],[1.00006,1.00006])